#Data Preparation

Install dan import package

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

Read Dataset

In [17]:
data_restaurant="https://docs.google.com/spreadsheets/d/e/2PACX-1vTyCBQxA1HyRnQWj-RXf-_5DXKn3L90Td0aO5r8DsTtE7pO3IQbPVvcuhfaCmJvTwiD_Hhvw34Xzbya/pub?gid=2129966204&single=true&output=csv"
data_makanminum="https://docs.google.com/spreadsheets/d/e/2PACX-1vTyCBQxA1HyRnQWj-RXf-_5DXKn3L90Td0aO5r8DsTtE7pO3IQbPVvcuhfaCmJvTwiD_Hhvw34Xzbya/pub?gid=477290477&single=true&output=csv"

# membaca kedua sheet ke dalam DataFrame
data_resto = pd.read_csv(data_restaurant)
data_makmin = pd.read_csv(data_makanminum)

print(data_resto.head())
print(data_makmin.head())

  resto_id                                      nama_restoran  \
0   RT01J1                Mie Bandung Chinese Food, Ngampilan   
1   RT02J1  Soto Tauco Pekalongan Bu Iswi, Jalan Bimokurdo...   
2   RT03J1                  Bakso Urat dan Mie Ayam "Pak Mir"   
3   RT04J1  Soto Ayam Kampung Dan Soto Daging Sapi Pak Gendut   
4   RT05J1                                          Toba Tabo   

  preferensi_makanan lokasi_restoran  Harga_Rata-Rata_Makanan_di_Toko (Rp)  \
0            Chinese         3.23 km                               39000.0   
1          Indonesia         3.01 km                               19000.0   
2          Indonesia         1.44 km                               18000.0   
3          Indonesia         0.83 km                               22000.0   
4            Chinese         2.13 km                               40000.0   

   rating_toko jenis_suasana toko_sering_diskon_(Ya/Tidak)  \
0          4.7        Santai                         Tidak   
1          3.7  

Memilih kolom yang digunakan

In [18]:
data_makmin.columns = data_makmin.columns.str.strip()

columns_to_select = ['makanan/minuman', 'kategori', 'deskripsi_rasa']
makanan_data = data_makmin[columns_to_select]

print("\nDataset dengan kolom terpilih:")
print(makanan_data.head())


Dataset dengan kolom terpilih:
  makanan/minuman       kategori                               deskripsi_rasa
0    Sate Klathak  Makanan Berat                   Gurih, pedas, juicy, smoky
1    Soto Kambing  Makanan Berat  Gurih, pedas, rempah, segar, lezat, berkuah
2    Sate Kambing  Makanan Berat      Gurih, pedas, juicy, smoky, asin, manis
3     Wedang Uwuh        Minuman                 Manis, hangat, rempah, segar
4          Bakpia        Jajanan                 Manis, lembut, gurih, kacang


In [19]:
#memilih kolom sheet data restoran
data_resto.columns = data_resto.columns.str.strip()

columns_select = ['nama_restoran', 'rating_toko', 'variasi_makanan']
restoran_data = data_resto[columns_select]

print("\nDataset dengan kolom terpilih:")
print(restoran_data.head())


Dataset dengan kolom terpilih:
                                       nama_restoran  rating_toko  \
0                Mie Bandung Chinese Food, Ngampilan          4.7   
1  Soto Tauco Pekalongan Bu Iswi, Jalan Bimokurdo...          3.7   
2                  Bakso Urat dan Mie Ayam "Pak Mir"          3.6   
3  Soto Ayam Kampung Dan Soto Daging Sapi Pak Gendut          3.8   
4                                          Toba Tabo          4.7   

                                     variasi_makanan  
0  Bakmi, Nasi Goreng, Mie Ayam, Kwetiaw, Bakso P...  
1  Bakso, Soto Tauco, Lontong, Es Teh, Es Jeruk, ...  
2         Bakso, Sop Ayam, Jajanan, Es Jeruk, Es Teh  
3  Soto Daging, Soto Ayam, Perkedel, Es Teh, Es J...  
4  Sate Babi, Babi Kecap, Bakmi Babi, Kopi, Es Ko...  


#Rekomendasi makanan berdasarkan kategori dan deskripsi rasa

In [20]:
# Combine category and taste for content-based filtering
makanan_data['Features'] = makanan_data['kategori'] + ' ' + makanan_data['deskripsi_rasa']

# Mengganti nilai NaN dengan string kosong
makanan_data['Features'] = makanan_data['Features'].fillna('')

# Encode the text data using TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(makanan_data['Features'])

In [21]:
# TensorFlow model
class ContentBasedRecommender(tf.keras.Model):
    def __init__(self):
        super(ContentBasedRecommender, self).__init__()
        self.dense1 = tf.keras.layers.Dense(256, activation='relu')
        self.dense2 = tf.keras.layers.Dense(128, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(64, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense5 = tf.keras.layers.Dense(len(makanan_data), activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dropout(x)
        return self.dense5(x)

In [22]:
# Input and output for training
X = tfidf_matrix.toarray()
y = tf.keras.utils.to_categorical(range(len(makanan_data)), len(makanan_data))

# Inisialisasi model
model = ContentBasedRecommender()

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
# Train the model
model.fit(X, y, epochs=1000, verbose=2)

Epoch 1/1000
11/11 - 2s - loss: 5.7779 - accuracy: 0.0000e+00 - 2s/epoch - 180ms/step
Epoch 2/1000
11/11 - 0s - loss: 5.7678 - accuracy: 0.0062 - 114ms/epoch - 10ms/step
Epoch 3/1000
11/11 - 0s - loss: 5.7576 - accuracy: 0.0155 - 70ms/epoch - 6ms/step
Epoch 4/1000
11/11 - 0s - loss: 5.7502 - accuracy: 0.0124 - 75ms/epoch - 7ms/step
Epoch 5/1000
11/11 - 0s - loss: 5.7273 - accuracy: 0.0093 - 62ms/epoch - 6ms/step
Epoch 6/1000
11/11 - 0s - loss: 5.6955 - accuracy: 0.0093 - 61ms/epoch - 6ms/step
Epoch 7/1000
11/11 - 0s - loss: 5.6345 - accuracy: 0.0093 - 64ms/epoch - 6ms/step
Epoch 8/1000
11/11 - 0s - loss: 5.5407 - accuracy: 0.0124 - 56ms/epoch - 5ms/step
Epoch 9/1000
11/11 - 0s - loss: 5.4283 - accuracy: 0.0248 - 75ms/epoch - 7ms/step
Epoch 10/1000
11/11 - 0s - loss: 5.3070 - accuracy: 0.0155 - 54ms/epoch - 5ms/step
Epoch 11/1000
11/11 - 0s - loss: 5.1778 - accuracy: 0.0248 - 51ms/epoch - 5ms/step
Epoch 12/1000
11/11 - 0s - loss: 5.0220 - accuracy: 0.0342 - 56ms/epoch - 5ms/step
Epoch 1

In [24]:
def recommend_makanan(category, taste):
    # Create the feature vector for the selected category and taste
    selected_features = category + ' ' + taste
    selected_vector = tfidf.transform([selected_features]).toarray()

    # Predict similarity scores
    predictions = model.predict(selected_vector)

    # Get the top 3 recommendations
    recommended_indices = predictions[0].argsort()[-10:][::-1]
    recommended_makanan = makanan_data.iloc[recommended_indices]['makanan/minuman'].values
    return recommended_makanan

inputan Dropdown button & free text

In [25]:
#buat sesuai kebutuhan kategori (Makanan Berat, Makanan Ringan, Jajanan, Minuman)
category_input = 'Makanan Berat'

In [26]:
#inputan free text
taste_input = 'Gurih, pedas'

In [27]:
recommended_makanan = recommend_makanan(category_input, taste_input)
print(f"Rekomendasi makanan untuk kategori '{category_input}' dengan rasa '{taste_input}':")
for makanan in recommended_makanan:
    print(f"- {makanan}")


1/1 [==============================] - 0s 340ms/step
Rekomendasi makanan untuk kategori 'Makanan Berat' dengan rasa 'Gurih, pedas':
- Nasi Pecel
- Gado-gado 
- Lotek
- Karedok
- Oseng Kangkung 
- Empal Bacem
- Empal Goreng
- Mie Aceh
- Sate kronyos
- Sate Ayam


#Rekomendasi restoran berdasarkan makanan yang dipilih



In [28]:
def recommend_restaurants_for_selected_food(selected_food):
    recommended_restaurants = []

    for i, row in restoran_data.iterrows():
        if selected_food in row['variasi_makanan']:
            recommended_restaurants.append((row['nama_restoran'], row['rating_toko'], row['variasi_makanan']))

    return recommended_restaurants

Cosine Similarity

In [29]:
# Replace NaN values with empty string
restoran_data['variasi_makanan'].fillna('', inplace=True)

# TF-IDF matrix for restaurant data
tfidf_matrix_restaurant = tfidf.transform(restoran_data['variasi_makanan'])

# Compute cosine similarity between food and restaurant TF-IDF matrices
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix_restaurant)

def recommend_restaurants_for_food_containing_keyword(keyword, cosine_sim, makanan_data, restoran_data):
    # Find the indices of foods that contain the keyword
    food_indices = makanan_data[makanan_data['makanan/minuman'].str.contains(keyword, case=False, na=False)].index.tolist()

    # Prepare the list of recommended restaurants with cosine similarity scores
    recommended_restaurants_with_scores = []

    for food_index in food_indices:
        # Compute and sort cosine similarity scores for each food
        sim_scores = list(enumerate(cosine_sim[food_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get top 3 similar restaurants with their cosine similarity scores for each food
        top_similar_restaurants = sim_scores[:5]

        for idx, score in top_similar_restaurants:
            recommended_restaurants_with_scores.append((restoran_data.iloc[idx]['nama_restoran'],
                                                        score, makanan_data.iloc[food_index]['makanan/minuman']))

    # Sort the final list by cosine similarity scores
    recommended_restaurants_with_scores.sort(key=lambda x: x[1], reverse=True)

    # Get unique recommendations, avoiding duplicates
    unique_recommendations = []
    seen_restaurants = set()
    for rec in recommended_restaurants_with_scores:
        if rec[0] not in seen_restaurants:
            seen_restaurants.add(rec[0])
            unique_recommendations.append(rec)

    return unique_recommendations[:5]

<ipython-input-29-59c2aa14ce45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restoran_data['variasi_makanan'].fillna('', inplace=True)


In [30]:
selected_food = 'babi'
recommended_restaurants_with_scores = recommend_restaurants_for_food_containing_keyword(selected_food, cosine_sim, makanan_data, restoran_data)

print(f"Rekomendasi restoran yang menyediakan makanan/minuman yang mengandung kata '{selected_food}' dengan skor kesamaan kosinus:")
for nama_restoran, score, variasi_makanan in recommended_restaurants_with_scores:
    print(f"- {nama_restoran} (skor kesamaan: {score:.2f}) menyediakan {variasi_makanan}")

Rekomendasi restoran yang menyediakan makanan/minuman yang mengandung kata 'babi' dengan skor kesamaan kosinus:
- Warung Lesehan Pak Kirun (skor kesamaan: 0.52) menyediakan Sate Babi 
- Madam Tan Indonesian Food (skor kesamaan: 0.43) menyediakan Sate Babi 
- Ayam Pedas Artomoro (skor kesamaan: 0.42) menyediakan Sate Babi 
- Waroeng spesial sambal "SS" Jakal KM 12 (skor kesamaan: 0.41) menyediakan Sate Babi 
- Katombo Sulawesi Grilled Fish (skor kesamaan: 0.40) menyediakan Sate Babi 
